In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import denumpyize, numpyize, get_template_live_config_vanilla, candidate_to_live_config, calc_spans, \
    analyze_fills, create_xk, round_dynamic, determine_config_type, round_values
from njit_funcs import calc_emas, calc_long_scalp_entry, calc_long_close_grid, round_, calc_bankruptcy_price, calc_diff, \
    calc_new_psize_pprice, qty_to_cost
from procedures import dump_live_config, load_live_config, add_argparse_args, prepare_backtest_config, make_get_filepath
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'XMRUSDT'
        self.market_type = 'futures'
        self.user = 'user_name'
        self.start_date = '2021-04-01'
        self.end_date = '2021-08-23'
        self.starting_balance = 1000.0
        self.starting_configs = ''
        self.base_dir = 'backtests'
config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'timestamp': timestamps, 'qty': qtys, 'price': prices}).set_index('timestamp')
df.price.iloc[::100].plot()

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "hand_tuned_scalp",
    "logging_level": 0,
    "long": {"enabled": True,
             "primary_ddown_factor": 1.15,
             "primary_grid_spacing":  0.028,
             "primary_initial_qty_pct":  0.01,
             "primary_pbr_limit": 0.5,
             "primary_grid_spacing_pbr_weighting": [0.045, 0.045],
             "secondary_ddown_factor": 2.0,
             "secondary_grid_spacing":  0.18,
             "secondary_pbr_limit_added":  0.5,
             "min_markup": 0.001,
             "markup_range": 0.009,
             "n_close_orders": 10},
    "shrt": {"enabled": True,
             "primary_ddown_factor": 1.15,
             "primary_grid_spacing":  0.028,
             "primary_initial_qty_pct":  0.01,
             "primary_pbr_limit": 0.6,
             "primary_grid_spacing_pbr_weighting": [0.065, 0.065],
             "secondary_ddown_factor": 2.0,
             "secondary_grid_spacing":  0.18,
             "secondary_pbr_limit_added":  0.5,
             "min_markup": 0.001,
             "markup_range": 0.009,
             "n_close_orders": 10}
}

# round for better human readability
hand_tuned = round_values(hand_tuned, 4)
config['starting_balance'] = 1000.0
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f'tmp/hand_tuned_scalp.json'))
hand_tuned

In [ ]:
sts = time()
fills, info = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
sample_size_ms = dataslice[1][0] - dataslice[0][0]
fdf, analysis = analyze_fills(fills, result, dataslice[0][0], dataslice[-1][0])
pprint.pprint(analysis)
fdf

In [ ]:
plot_fills(df, fdf)

In [ ]:
fdf.tail(60)

In [ ]:
fdf.balance.plot()
fdf.equity.plot()

In [ ]:
fdf.psize.plot()

In [ ]:
periodic_gain_n_days = 5
groups = fdf.groupby(fdf.timestamp // (1000 * 60 * 60 * 24 * periodic_gain_n_days))
pgs = groups.pnl.sum() / groups.balance.first()
pgs = pgs.reindex(np.arange(pgs.index[0], pgs.index[-1])).fillna(0.0)
pg_mean = pgs.mean()
pg_std = pgs.std()
sharpe_ratio = pg_mean / pg_std if pg_std != 0.0 else -20.0
pgs.plot()
plt.plot([pgs.index[0], pgs.index[-1]], [pg_mean, pg_mean])
print(f'mean {pg_mean:.4f} std {pg_std:.4} sharpe ratio {sharpe_ratio:.4f}')
pgs

In [ ]:
# view grid
# long only

In [ ]:
xk = create_xk(config_to_test)
spot = xk['spot']
inverse = xk['inverse']
qty_step = xk['qty_step']
price_step = xk['price_step']
min_qty = xk['min_qty']
min_cost = xk['min_cost']
c_mult = xk['c_mult']
primary_initial_qty_pct = xk['primary_initial_qty_pct'][0]
primary_ddown_factor = xk['primary_ddown_factor'][0]
primary_grid_spacing = xk['primary_grid_spacing'][0]
primary_grid_spacing_pbr_weighting = xk['primary_grid_spacing_pbr_weighting'][0]
primary_pbr_limit = xk['primary_pbr_limit'][0]
secondary_ddown_factor = xk['secondary_ddown_factor'][0]
secondary_grid_spacing = xk['secondary_grid_spacing'][0]
secondary_pbr_limit_added = xk['secondary_pbr_limit_added'][0]
min_markup = xk['min_markup'][0]
markup_range = xk['markup_range'][0]
n_close_orders = xk['n_close_orders'][0]

balance = 1000.0
long_psize = 0.0
long_pprice = 0.0
highest_bid = prices[-1]
lowest_ask = prices[-1]
long_pfills = ((0.0, 0.0),)
do_long = True
max_leverage = 20

orders = []

print("qty          price          psize         pprice        pprice / price ratio    pbr        bkr diff   type")
for k in range(10):
    long_entry = calc_long_scalp_entry(
        balance, long_psize, long_pprice, long_pfills, highest_bid,
        spot, inverse, do_long, qty_step, price_step, min_qty, min_cost,
        c_mult, max_leverage, primary_initial_qty_pct, primary_ddown_factor,
        primary_grid_spacing, primary_grid_spacing_pbr_weighting, primary_pbr_limit,
        secondary_ddown_factor, secondary_grid_spacing, secondary_pbr_limit_added

    )
    long_psize, long_pprice = calc_new_psize_pprice(long_psize, long_pprice, long_entry[0], long_entry[1], qty_step)
    long_pcost = qty_to_cost(long_psize, long_pprice, inverse, c_mult)
    bkr_price = calc_bankruptcy_price(balance,
                          long_psize,
                          long_pprice,
                          0.0,
                          0.0,
                          inverse, c_mult)
    bkr_diff = calc_diff(bkr_price, long_entry[1]) if long_entry[1] != 0.0 else 1.0
    pbr = long_pcost / balance
    if long_entry[0] == 0.0:
        break
    print(f"{long_entry[0]: <12} {long_entry[1]: <14} {round_(long_psize, qty_step): <12}  {round_(long_pprice, price_step): <14}"
          f"{round(long_pprice / long_entry[1], 4): <23} {round(pbr, 4): <10} {round(bkr_diff, 6): <10} {long_entry[2]}")
    orders.append(long_entry)
print()
orders = denumpyize(orders)
primary_orders = [o for o in orders if 'primary' in o[2] or 'ientry' in o[2]]
secondary_orders = [o for o in orders if 'secondary' in o[2]]

print(f'primary grid span {(primary_orders[0][1] / primary_orders[-1][1] - 1) * 100:.4f}%')
print(f'primary + secondary grid span {(orders[0][1] / orders[-1][1] - 1) * 100:.4f}%')
#print('dist between grid entries', entry_prices[:-1] / entry_prices[1:])

In [ ]:
n_days_ = 14
nticks = len(df[df.index > (time() - 60 * 60 * 24 * n_days_) * 1000])
print('n ticks', nticks)
dfc = df.iloc[-nticks:]
primarydf = pd.DataFrame({p: [p, p] for p in [o[1] for o in primary_orders]}, index=[dfc.index[0], df.index[-1]])
plt.plot(dfc.index.values, dfc.price.values)
plt.plot(primarydf.index.values, primarydf.values, 'b-')
if secondary_orders:
    secondarydf = pd.DataFrame({p: [p, p] for p in [o[1] for o in secondary_orders]}, index=[dfc.index[0], df.index[-1]])
    plt.plot(secondarydf.index.values, secondarydf.values, 'r-')
